In [ ]:
!unzip tatrach-autoformer-main.zip

Archive:  tatrach-autoformer-main.zip
   creating: tatrach-autoformer-main/
  inflating: tatrach-autoformer-main/.gitignore  
   creating: tatrach-autoformer-main/dataset/
  inflating: tatrach-autoformer-main/dataset/Data_TaTrach.csv  
   creating: tatrach-autoformer-main/dataset/ETT-small/
  inflating: tatrach-autoformer-main/dataset/ETT-small/ETTh1.csv  
   creating: tatrach-autoformer-main/data_provider/
  inflating: tatrach-autoformer-main/data_provider/data_factory.py  
  inflating: tatrach-autoformer-main/data_provider/data_loader.py  
 extracting: tatrach-autoformer-main/data_provider/__init__.py  
  inflating: tatrach-autoformer-main/Dockerfile  
  inflating: tatrach-autoformer-main/environment.yml  
   creating: tatrach-autoformer-main/exp/
  inflating: tatrach-autoformer-main/exp/exp_basic.py  
  inflating: tatrach-autoformer-main/exp/exp_main.py  
 extracting: tatrach-autoformer-main/exp/__init__.py  
   creating: tatrach-autoformer-main/layers/
  inflating: tatrach-autoform

In [ ]:
import torch
print(torch.cuda.is_available())		 # 查看GPu设备是否可用
print(torch.cuda.device_count()) 		# 查看GPu设备数量
print(torch.cuda.get_device_name())   	# 查看当前GPu设备名称，默认设备id从0开始
print(torch.cuda.current_device())

True
1
NVIDIA L4
0


In [ ]:
!pip install reformer-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main#exp stands for experiments
import random
import numpy as np
from utils.tools import dotdict

# fix_seed = 2021
# random.seed(fix_seed)
# torch.manual_seed(fix_seed)
# np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')#模型id
parser.add_argument('--model', type=str, required=True, default='Autoformer',#选择模型
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')#数据类型
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')#数据文件夹路径
parser.add_argument('--data_path', type=str, default='mucnuoc_gio_preprocess.csv', help='data file')#具体文件
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')#预测类别
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')#不太懂 OT好像代表Output Target,要预测的单变量
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')#保存模型

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')#输入序列长度
parser.add_argument('--label_len', type=int, default=48, help='start token length')#这个label_len未完全搞懂
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')#输出序列长度

# model define
parser.add_argument('--bucket_size', type=int, default=4, help='for Reformer')#Reformer专用属性
parser.add_argument('--n_hashes', type=int, default=4, help='for Reformer')#Reformer专用属性
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')#encoder input size
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')#decoder input size
parser.add_argument('--c_out', type=int, default=7, help='output size')#输出长度
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')#dimension of model
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')#num of heads
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')#num of encoder layers
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')#num of decoder layers
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')#dimension of fcn
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')#窗口滑动平均数
parser.add_argument('--factor', type=int, default=1, help='attn factor')#attn factor不太理解
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)#是否在encoder里面使用知识蒸馏
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')#dropout
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')#time features encoding不太能get到
parser.add_argument('--activation', type=str, default='gelu', help='activation')#激活函数default=gelu
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')#encoder的output_attention是否输出
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')#是否预测未见的未来数据,也就是是否进行推理的意思

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')# num_workers是加载数据(batch)的线程数目
parser.add_argument('--itr', type=int, default=2, help='experiments times')#实验次数
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')#就是epoch
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')#bathsize
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')#patience: 当early stop被激活(如发现loss相比上一个epoch训练没有下降)，则经过patience个epoch后停止训练
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')#lr
parser.add_argument('--des', type=str, default='test', help='exp description')#test
parser.add_argument('--loss', type=str, default='mse', help='loss function')#loss is mse
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')#adjust learning-rate
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)#使用自动混合精度训练

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')

# args = parser.parse_args()
args = dotdict()
args.target = 'OT'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False
# if args.use_gpu and args.use_multi_gpu: #是否使用多卡的判断
#     args.dvices = args.devices.replace(' ', '')
#     device_ids = args.devices.split(',')
#     args.device_ids = [int(id_) for id_ in device_ids]
#     args.gpu = args.device_ids[0]
args.freq = 'h'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.seq_len = 24
args.label_len = 1
args.pred_len = 72
args.e_layers = 6
args.d_layers = 5
args.n_heads = 32
args.factor = 1
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.distil = True
args.output_attention = False
args.patience= 3
args.learning_rate = 1e-4
args.batch_size = 32
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'

args.train_epochs = 10
args.is_training = True
args.enc_in = 1
args.dec_in = 1
args.c_out = 1
args.target = "q64"
args.root_path = './dataset/'
args.data_path ='mucnuoc_gio_preprocess.csv'
args.model_id='q64'
args.model = 'Autoformer'
#args.model = 'Informer'
args.data = 'custom'
args.features = 'S'

print('Args in experiment:')
print(args)



Exp = Exp_Main

Args in experiment:
{'target': 'q64', 'des': 'Exp', 'dropout': 0.05, 'num_workers': 10, 'gpu': 0, 'lradj': 'type1', 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False, 'freq': 'h', 'checkpoints': './checkpoints/', 'bucket_size': 4, 'n_hashes': 4, 'seq_len': 24, 'label_len': 1, 'pred_len': 72, 'e_layers': 6, 'd_layers': 5, 'n_heads': 32, 'factor': 1, 'd_model': 512, 'itr': 1, 'd_ff': 2048, 'moving_avg': 25, 'distil': True, 'output_attention': False, 'patience': 3, 'learning_rate': 0.0001, 'batch_size': 32, 'embed': 'timeF', 'activation': 'gelu', 'use_amp': False, 'loss': 'mse', 'train_epochs': 10, 'is_training': True, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'root_path': './dataset/', 'data_path': 'mucnuoc_gio_preprocess.csv', 'model_id': 'q64', 'model': 'Autoformer', 'data': 'custom', 'features': 'S'}


In [ ]:
import h5py
import pickle
import os
from datetime import datetime
def save_model_simple(exp, args, save_dir="/content/save_model"):
    """Lưu model đơn giản: H5 + PKL files"""

    # Tạo folder
    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    print(f"💾 Saving model to: {save_dir}")

    # 1. Lưu model weights (.h5)
    model_path = os.path.join(save_dir, f"autoformer_model_window_{args.target}.h5")
    with h5py.File(model_path, 'w') as f:
        # Lưu weights
        for key, tensor in exp.model.state_dict().items():
            if isinstance(tensor, torch.Tensor):
                f.create_dataset(key, data=tensor.detach().cpu().numpy())

    print(f"✅ Model saved: {os.path.basename(model_path)}")

    # 2. Lưu scaler (.pkl)
    print("Debugging scaler save...")
    scaler_saved = False

    # Thử cách 2: Lấy từ data loader khác
    if not scaler_saved:
        try:
            print("   Trying method 2: from data_loader directly...")
            train_data, train_loader = exp._get_data(flag='train')

            # Kiểm tra các thuộc tính có thể chứa scaler
            for attr_name in ['scaler', 'data_scaler', 'transform', 'normalizer']:
                if hasattr(train_data, attr_name):
                    scaler = getattr(train_data, attr_name)
                    print(f"   Found scaler in attribute: {attr_name}")

                    scaler_path = os.path.join(save_dir, f"scaler_{args.target}_{timestamp}.pkl")
                    with open(scaler_path, 'wb') as f:
                        pickle.dump(scaler, f)
                    print(f"✅ Scaler saved: {os.path.basename(scaler_path)}")
                    scaler_saved = True
                    break

        except Exception as e:
            print(f"   Method 2 failed: {e}")

    # 3. Lưu config (.pkl)
    config = {
        'model': args.model,
        'target': args.target,
        'seq_len': args.seq_len,
        'pred_len': args.pred_len,
        'd_model': args.d_model,
        'n_heads': args.n_heads,
        'e_layers': args.e_layers,
        'd_layers': args.d_layers,
        'enc_in': args.enc_in,
        'dec_in': args.dec_in,
        'c_out': args.c_out,
        'features': args.features,
        'dropout': args.dropout,
        'factor': args.factor,
        'moving_avg': getattr(args, 'moving_avg', 25),
        'activation': args.activation,
        'embed': args.embed,
        'freq': args.freq
    }
    print(args.pred_len)
    config_path = os.path.join(save_dir, f"config_autoformer_window_{args.target}.pkl")
    with open(config_path, 'wb') as f:
        pickle.dump(config, f)

    print(f"✅ Config saved: {os.path.basename(config_path)}")
    print(f"📁 All files saved in: {save_dir}")

    return save_dir

In [ ]:
if args.is_training:
  for ii in range(args.itr):
      # setting record of experiments
      setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
          args.model_id,
          args.model,
          args.data,
          args.features,
          args.seq_len,
          args.label_len,
          args.pred_len,
          args.d_model,
          args.n_heads,
          args.e_layers,
          args.d_layers,
          args.d_ff,
          args.factor,
          args.embed,
          args.distil,
          args.des, ii)

      exp = Exp(args)  # set experiments
      print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
      exp.train(setting)

      print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
      exp.test(setting)

      if args.do_predict:
          print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
          exp.predict(setting, True)

      torch.cuda.empty_cache()
else:
  ii = 0
  setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                args.model,
                                                                                                args.data,
                                                                                                args.features,
                                                                                                args.seq_len,
                                                                                                args.label_len,
                                                                                                args.pred_len,
                                                                                                args.d_model,
                                                                                                args.n_heads,
                                                                                                args.e_layers,
                                                                                                args.d_layers,
                                                                                                args.d_ff,
                                                                                                args.factor,
                                                                                                args.embed,
                                                                                                args.distil,
                                                                                                args.des, ii)

  exp = Exp(args)  # set experiments
  print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
  exp.test(setting, test=1)
  torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : q64_Autoformer_custom_ftS_sl24_ll1_pl72_dm512_nh32_el6_dl5_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 33157
val 4680
test 9429
	iters: 100, epoch: 1 | loss: 0.0365545
	speed: 0.1197s/iter; left time: 1228.5124s
	iters: 200, epoch: 1 | loss: 0.0137743
	speed: 0.1173s/iter; left time: 1191.6078s
	iters: 300, epoch: 1 | loss: 0.0128247
	speed: 0.1166s/iter; left time: 1173.5936s
	iters: 400, epoch: 1 | loss: 0.0107844
	speed: 0.1157s/iter; left time: 1152.6943s
	iters: 500, epoch: 1 | loss: 0.0111502
	speed: 0.1151s/iter; left time: 1134.9631s
	iters: 600, epoch: 1 | loss: 0.0108958
	speed: 0.1148s/iter; left time: 1120.1758s
	iters: 700, epoch: 1 | loss: 0.0135873
	speed: 0.1149s/iter; left time: 1110.4087s
	iters: 800, epoch: 1 | loss: 0.0093423
	speed: 0.1151s/iter; left time: 1100.6406s
	iters: 900, epoch: 1 | loss: 0.0141152
	speed: 0.1156s/iter; left time: 1093.2691s
	iters: 1000, epoch: 1 | loss: 0.0126289
	speed: 0.1157s

In [ ]:
print(args)

{'target': 'q64', 'des': 'Exp', 'dropout': 0.05, 'num_workers': 10, 'gpu': 0, 'lradj': 'type1', 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False, 'freq': 'h', 'checkpoints': './checkpoints/', 'bucket_size': 4, 'n_hashes': 4, 'seq_len': 24, 'label_len': 1, 'pred_len': 2, 'e_layers': 6, 'd_layers': 5, 'n_heads': 32, 'factor': 1, 'd_model': 512, 'itr': 1, 'd_ff': 2048, 'moving_avg': 25, 'distil': True, 'output_attention': False, 'patience': 3, 'learning_rate': 0.0001, 'batch_size': 32, 'embed': 'timeF', 'activation': 'gelu', 'use_amp': False, 'loss': 'mse', 'train_epochs': 10, 'is_training': True, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'root_path': './dataset/', 'data_path': 'mucnuoc_gio_preprocess.csv', 'model_id': 'q64', 'model': 'Autoformer', 'data': 'custom', 'features': 'S'}


In [ ]:
print("\n" + "="*50)
save_model_simple(exp, args)
print("="*50 + "\n")


💾 Saving model to: /content/save_model
✅ Model saved: autoformer_model_window_q64.h5
Debugging scaler save...
   Trying method 2: from data_loader directly...
train 33157
   Found scaler in attribute: scaler
✅ Scaler saved: scaler_q64_20250614_053727.pkl
72
✅ Config saved: config_autoformer_window_q64.pkl
📁 All files saved in: /content/save_model

